In [1]:
%pwd

'D:\\Work\\notebook\\metricstudy\\notebook'

In [2]:
BTC_ETH = "BTC_ETH"

poloniex_url_template = "https://poloniex.com/public?command=returnTradeHistory&currencyPair=%s&start=%s&end=%s"


In [3]:
#currency_pair: currency pair expression which is supported in poloniex, to: unixtime value
def get_url(currency_pair, end_unix):
    sixmonths = 60*60*24*30*6
    start_unix = end_unix - sixmonths
    url = poloniex_url_template%(currency_pair, start_unix, end_unix)
    return url

In [4]:
import time
import datetime

def now():
    #now = int(time.gmtime())
    now = datetime.datetime.utcnow()
    now = int(time.mktime(now.timetuple()))
    print(type(now))
    return now
#1493047494
now()

<class 'int'>


1493019110

In [5]:
url = get_url(BTC_ETH, now())
print (url)

<class 'int'>
https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1477467111&end=1493019111


In [6]:
import requests
r = requests.get(url)
js = r.json()
print(js[:2])
print(js[-2:])

[{'amount': '100.76937592', 'type': 'buy', 'tradeID': 23046136, 'globalTradeID': 111224587, 'rate': '0.03980534', 'date': '2017-04-24 07:31:45', 'total': '4.01115927'}, {'amount': '0.26627370', 'type': 'sell', 'tradeID': 23046135, 'globalTradeID': 111224585, 'rate': '0.03980532', 'date': '2017-04-24 07:31:45', 'total': '0.01059910'}]
[{'amount': '0.03829461', 'type': 'buy', 'tradeID': 22996138, 'globalTradeID': 110532237, 'rate': '0.03916990', 'date': '2017-04-22 20:34:45', 'total': '0.00149999'}, {'amount': '0.01917913', 'type': 'buy', 'tradeID': 22996137, 'globalTradeID': 110532236, 'rate': '0.03910500', 'date': '2017-04-22 20:34:45', 'total': '0.00074999'}]


In [7]:
lastdate = js[-1]['date']
print(lastdate)


2017-04-22 20:34:45


In [8]:
import datetime

def get_gmttime(lastdate):
    #print(datetime.datetime.strptime(lastdate, "%Y-%m-%d %H:%M:%S"))
    endtime = int(datetime.datetime.strptime(lastdate, "%Y-%m-%d %H:%M:%S").timestamp())+(9*60*60)
    #print(endtime)
    return endtime
#1493015351
#1492857459

In [9]:
endtime = get_gmttime(lastdate)

url2 = get_url(BTC_ETH, endtime)
print (url2)

https://poloniex.com/public?command=returnTradeHistory&currencyPair=BTC_ETH&start=1477341285&end=1492893285


In [10]:
import requests
r = requests.get(url2)
js2 = r.json()
print(js2[:2])
print(js2[-2:])

[{'amount': '0.53783626', 'type': 'buy', 'tradeID': 22996139, 'globalTradeID': 110532238, 'rate': '0.03918300', 'date': '2017-04-22 20:34:45', 'total': '0.02107403'}, {'amount': '0.03829461', 'type': 'buy', 'tradeID': 22996138, 'globalTradeID': 110532237, 'rate': '0.03916990', 'date': '2017-04-22 20:34:45', 'total': '0.00149999'}]
[{'amount': '71.51430000', 'type': 'buy', 'tradeID': 22946141, 'globalTradeID': 109768352, 'rate': '0.03917400', 'date': '2017-04-21 10:27:03', 'total': '2.80150118'}, {'amount': '0.00662976', 'type': 'buy', 'tradeID': 22946140, 'globalTradeID': 109768351, 'rate': '0.03913285', 'date': '2017-04-21 10:27:03', 'total': '0.00025944'}]


In [36]:
import MySQLdb

In [37]:
len(js)

50000

In [38]:
len(js2)

50000

In [39]:
db = MySQLdb.connect(db='poloniex', user='polo', passwd='polo')

In [40]:
curr = db.cursor()

In [43]:
for item in js[:2]:
    #print (item)
    value = "%d,%d,%s,%s,%s,'%s','%s'"%(item['globalTradeID'], item['tradeID'], item['total'], item['rate'], item['amount'], item['type'], item['date'])
    
    try:
        query = "insert into btc_eth values(%s)"%(value)
        print (query)
        curr.execute(query)
    except (MySQLdb.Error, MySQLdb.Warning) as e:
        print ("MySQL Error [%d]: %s" % (e.args[0], e.args[1]))
    except IndexError:
        print ("MySQL Error: %s" % str(e))

insert into btc_eth values(111224587,23046136,4.01115927,0.03980534,100.76937592,'buy','2017-04-24 07:31:45')
MySQL Error [1062]: Duplicate entry '111224587' for key 'PRIMARY'
insert into btc_eth values(111224585,23046135,0.01059910,0.03980532,0.26627370,'sell','2017-04-24 07:31:45')
MySQL Error [1062]: Duplicate entry '111224585' for key 'PRIMARY'


In [26]:
for item in js:
    print (','.join([str(s) for s in item.values()]))
    limit -= 1
    if limit == 0:
        break

100.76937592,buy,23046136,111224587,0.03980534,2017-04-24 07:31:45,4.01115927
0.26627370,sell,23046135,111224585,0.03980532,2017-04-24 07:31:45,0.01059910
8.13710240,buy,23046134,111224564,0.03980536,2017-04-24 07:31:41,0.32390029
0.04289760,buy,23046133,111224563,0.03980533,2017-04-24 07:31:41,0.00170755
0.25122257,sell,23046132,111224479,0.03980534,2017-04-24 07:31:17,0.00999999
0.08995801,buy,23046131,111224476,0.03980536,2017-04-24 07:31:14,0.00358081
2.91004199,buy,23046130,111224475,0.03980533,2017-04-24 07:31:14,0.11583518
0.29000000,sell,23046129,111224471,0.03980532,2017-04-24 07:31:13,0.01154354
0.28640127,buy,23046128,111224382,0.03980534,2017-04-24 07:30:57,0.01140029
0.15815875,sell,23046127,111224379,0.03980532,2017-04-24 07:30:57,0.00629555
